In [ ]:
############
# https://stackoverflow.com/questions/69800749/using-python-to-get-holdings-and-allocation-of-etfs
# conda env create -f environment.yml
# conda install -f environment.yml
# conda update -n base conda
# conda env update --file environment.yml --prune
# print("https://www.etfdb.com")
print("https://www.etfrc.com/funds/overlap.php")
TRADITIONAL = 'VYM VOT MGK VIG VGT VOO VDE VCR VHT VFH VUG'
# "AIRR CLSE CRPT HTUS IGM KBWP MGK PAVE PKB RSHO SLX SPSM VCR VDE VFH VGT VHT VIG VOO VOT VUG VYM XBI XHB"
ROTH = 'SPYG SCHG QQQ VGT CRPT IGM PAVE PKB XHB' # Roth 2024.03.14. 04.01 -IVW VOO MGK VTI IUSG VUG

BASE = 'VOT VOO VTI FLMVX FXAIX'
CONSTRUCTION = 'AIRR PKB XHB ITB'
GROWTH = 'MGK VGT VUG QQQ IVW SCHG SPYG IUSG FCNKX JDMNX MITJX VIGIX'
DIV = 'VYM VIG'
ENERGY = 'AMZA URNM VDE'
INSURANCE = 'KBWP'
SECTORS = 'VOX VCR VDC VFH VHT VIS VAW VPU RSHO PRN'
INTERNATIONALS = 'VEU VSS VEA VWO VGK VPL VNQI VXUS VT'
CRYPTO = 'CRPT'# DAPP BITQ BKCH SATO WGMI IBLC # BITX (2023-06-27) GBTC IBIT (2024-1-11)
INFRA = 'PAVE'# IGF IFRA NFRA EMIF RBLD
AI = 'IGM IYW FTEC FDN'

BENCH = 'VOO CRPT SPSM PAVE '# KBWP PAVE IGM VGT QMOM URNM AMZA PKB=XHB SLX
TEST = BENCH
TEST = BENCH + 'CLSE XBI IAI KCE PSP XMMO XSMO UTES AIRR PKB KBWP XHB HTUS' # HRTS CBLS USE GCAD SLX VDE
start = '2020-11-03'

In [ ]:
# Import libraries and pull data
import datetime
import warnings
import re
import json
import textwrap
import numpy as np
import pandas as pd
# import pandas_ta as ta
import yfinance as yf
#import quantstats as qs
import matplotlib.pyplot as plt
import seaborn as sns
import cufflinks as cf
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from ipywidgets import widgets, interact, interactive
from scipy import stats
#from sklearn.preprocessing import MinMaxScaler, StandardScaler
#from keras.models import Sequential
#from keras.layers import Dense, LSTM, Activation
from IPython.display import display, clear_output, HTML
from yahooquery import Ticker, Screener
from pyvis import network as net
import networkx as nx

cf.go_offline()
pio.renderers.default = 'iframe'
plt.style.use('dark_background')
warnings.filterwarnings("ignore", category=FutureWarning, module="yfinance")
warnings.filterwarnings("ignore", category=FutureWarning, module="cufflinks")
warnings.filterwarnings("ignore", "is_categorical_dtype")
warnings.filterwarnings("ignore", "use_inf_as_na")
warnings.filterwarnings("ignore", "warn_singular")
warnings.simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
template = 'plotly_dark'
pd.options.display.float_format = "{:,.2f}".format

EPOCH=64
EPOCH=32
NOP = 20000 # Number of random portfolio sets
risk_free_return = .05
model = 'Sharpe'
categories = {
    "ai": AI,
    "base": BASE,
    "construction": CONSTRUCTION,
    "crypto": CRYPTO,
    "div": DIV,
    "energy": ENERGY,
    "growth": GROWTH,
    "infra": INFRA,
    "insurance": INSURANCE,
    "internationals": INTERNATIONALS,
    "sectors": SECTORS,
}

In [ ]:
def calc_positions_ichimoku(d):
    global idx
    columns = d['Low'].columns
    # Adding signals for buy/ sell
    sgn = pd.DataFrame(np.where((d['tenkan_sen'] > d['kijun_sen']) & (d['Close'] > d['tenkan_sen']), 1, 0), columns=columns, index=d.index)
    sgn.columns = pd.MultiIndex.from_product([['signal_ichimoku'], columns])
    pos = sgn.diff().fillna(0)
    pos.columns = pd.MultiIndex.from_product([['position_ichimoku'], columns])
    idx.drop(columns='signal_ichimoku', inplace=True, errors='ignore')
    idx.drop(columns='position_ichimoku', inplace=True, errors='ignore')
    idx = idx.join(pos)
    idx = idx.join(sgn)

def calc_positions_rsi(d):
    global idx
    columns = d['Low'].columns
    # Adding signals for buy/ sell
    indicator='RSI'
    period=42
    sgn = pd.DataFrame(np.where(d[indicator].rolling(window=period).mean() - idx[indicator] - .3 < 0, 0, 1), columns=columns, index=d.index)
    sgn.columns = pd.MultiIndex.from_product([['signal_rsi'], columns])
    pos = sgn.diff().fillna(0)
    pos.columns = pd.MultiIndex.from_product([['position_rsi'], columns])
    idx.drop(columns='signal_rsi', inplace=True, errors='ignore')
    idx.drop(columns='position_rsi', inplace=True, errors='ignore')
    idx = idx.join(pos)
    idx = idx.join(sgn)
    
def calc_positions_ema(d):
    global idx
    columns = d['Low'].columns
    # Adding signals for buy/ sell
    sgn = pd.DataFrame(np.where(d['EMAW'] > d['EMAM'], 1, 0), columns=columns, index=d.index)
    sgn.columns = pd.MultiIndex.from_product([['signal_ema'], columns])
    pos = sgn.diff().fillna(0)
    pos.columns = pd.MultiIndex.from_product([['position_ema'], columns])
    idx.drop(columns='signal_ema', inplace=True, errors='ignore')
    idx.drop(columns='position_ema', inplace=True, errors='ignore')
    idx = idx.join(pos)
    idx = idx.join(sgn)

def price_bands(period = 42, multiplier = 1.8): # Calculate price bands
    global idx
    
    columns = idx['Low'].columns
    
    std = idx['Adj Close'].rolling(window=period).std()
    std.columns = pd.MultiIndex.from_product([['STD'], columns])
    idx = idx.join(std)
    
    ma = idx['Adj Close'].rolling(window=period).mean()
    ma.columns = pd.MultiIndex.from_product([['MA'], columns])
    idx = idx.join(ma)
    
    uband = idx['MA'] + multiplier * idx['STD']
    uband.columns = pd.MultiIndex.from_product([['UBAND'], columns])
    idx = idx.join(uband)
    
    lband = idx['MA'] - multiplier * idx['STD']
    lband.columns = pd.MultiIndex.from_product([['LBAND'], columns])
    idx = idx.join(lband)

    emaw = idx['Adj Close'].ewm(span=5, adjust=False).mean()
    emaw.columns = pd.MultiIndex.from_product([['EMAW'], columns])
    idx = idx.join(emaw)
    
    emam = idx['Adj Close'].ewm(span=21, adjust=False).mean()
    emam.columns = pd.MultiIndex.from_product([['EMAM'], columns])
    idx = idx.join(emam)

    # strength = 2 * (idx['Adj Close'] - idx['UBAND']) / (idx['UBAND'] - idx['LBAND']) + 1
    # strength.columns = pd.MultiIndex.from_product([['strength'], columns])
    # idx = idx.join(strength)
    
    # Moving Average Convergence Divergence
    mdl = idx['EMAM'] - idx['EMAW']
    sgl = mdl.ewm(span=16, adjust=False).mean()
    mdh = sgl - mdl

    # mdl.columns = pd.MultiIndex.from_product([['MACD_Line'], columns])
    # sgl.columns = pd.MultiIndex.from_product([['Signal_Line'], columns])
    mdh.columns = pd.MultiIndex.from_product([['MACD'], columns])
    
    # idx = idx.join(mdl)
    # idx = idx.join(sgl)
    idx = idx.join(mdh)

    # Relative Strength Index
    rsi = calculate_rsi(idx)/25-2
    rsi.columns = pd.MultiIndex.from_product([['RSI'], columns])
    idx = idx.join(rsi)
    
    idx = calc_ichimoku(idx)
    calc_positions_ichimoku(idx)
    calc_positions_ema(idx)
    calc_positions_rsi(idx)

    summary['UBAND'] = idx['UBAND'].iloc[-1]
    summary['Adj Close'] = idx['Adj Close'].iloc[-1]
    summary['LBAND'] = idx['LBAND'].iloc[-1]
    # summary['RSI'] = idx['RSI'].iloc[-1]
    # summary['MACD'] = idx['MACD'].iloc[-1]

    for indicator in ['MACD', 'RSI']:
        summary[f"{indicator}_prc"] = 100 - (idx[indicator] < idx[indicator].iloc[-1]).mean() * 100

def calculate_rsi(data, window=10):
    delta = data['Adj Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

def fillcol(d):
    np.where(d['senkou_span_a'] > d['senkou_span_b'], 'green', 'red')
    # np.where(df['A'] > 1, df['B'], 0)
    
#Calculate ichimoku 
def calc_ichimoku(d, tenkan_window=9, kijun_window=26, senkou_span_b_window=52, cloud_displacement=26, chikou_shift=-26):
    last_date = d['Low'].dropna().index[-1]
    if type(d['Low']) == pd.core.frame.DataFrame:
        columns = d['Low'].columns
        # Tenkan-sen (Conversion Line): (9-period high + 9-period low)/2))
        nine_period_high = d['High'].rolling(window= tenkan_window).max()
        nine_period_low = d['Low'].rolling(window= tenkan_window).min()
        tenkan_sen = (nine_period_high + nine_period_low) /2
        tenkan_sen.columns = pd.MultiIndex.from_product([['tenkan_sen'], columns])
        d = d.join(tenkan_sen)

        # Kijun-sen (Base Line): (26-period high + 26-period low)/2))
        period26_high = d['High'].rolling(window=kijun_window).max()
        period26_low = d['Low'].rolling(window=kijun_window).min()
        kijun_sen = (period26_high + period26_low) / 2
        kijun_sen.columns = pd.MultiIndex.from_product([['kijun_sen'], columns])
        d = d.join(kijun_sen)

    else:
        # Step 2: Create a new date range for the next 26 business days
        new_dates = pd.date_range(start=last_date+pd.Timedelta(days=1), periods=kijun_window, freq=pd.offsets.BDay())
        # Step 3: Convert the DatetimeIndex to a DateIndex (i.e., remove the time part)
        new_dates = new_dates.date  # This will convert the index to date type
        # Step 4: Create a new dataframe with the new business dates (as date type index)
        new_data = pd.DataFrame(index=new_dates)
        # Step 5: Append the new rows to the original dataframe
        d = pd.concat([d, new_data])

        # Senkou Span A (Leading Span A): (Conversion Line + Base Line)/2))
        d['senkou_span_a'] = ((d['tenkan_sen'] + d['kijun_sen']) / 2).shift(cloud_displacement)

        # Senkou Span B (Leading Span B): (52-period high + 52-period low)/2))
        period52_high = d['High'].rolling(window=senkou_span_b_window).max()
        period52_low = d['Low'].rolling(window=senkou_span_b_window).min()
        d['senkou_span_b'] = ((period52_high + period52_low) / 2).shift(cloud_displacement)

        # The most current closing price plotted 26 time periods behind (optional)
        d['chikou_span'] = d['Close'].shift(-chikou_shift)
    return d
    
def plot2_ichimoku(d, ticker_name='', INCREASING_COLOR='green', DECREASING_COLOR='red', show_rangeslider=False):
    fig = go.Figure(data=[go.Candlestick(x=d.index, open=d.Open, high=d.High, low=d.Low, close=d.Close,
         increasing_line_color=INCREASING_COLOR, decreasing_line_color=DECREASING_COLOR, showlegend=False)])
    kw = {'x': d.index, 'type': 'scatter', 'mode':'lines', 'visible':'legendonly'}

    # Add TenkenSen to plot
    fig.add_trace(go.Scatter(y=d['tenkan_sen'], name='tenkan_sen', line=dict(color='#33BDFF', width=1), **kw))
    # Add kijun_sen to plot
    fig.add_trace(go.Scatter(y=d['kijun_sen'], name='kijun_sen', line=dict(color='#F1F316', width=1), **kw))

    kw = {'x': d.index, 'type': 'scatter', 'mode':'lines', 'showlegend': False}
    # Add senkou_span_a to plot
    # fig.add_trace(go.Scatter(y=d['senkou_span_a'], name='senkou_span_a', line=dict(color='#228B22', width=1), **kw))
    fig.add_trace(go.Scatter(y=np.where(d['senkou_span_a'] > d['senkou_span_b'], d['senkou_span_a'], d['senkou_span_b']), name='senkou_span_max', line=dict(color='#228B22', width=0), **kw))
    fig.add_trace(go.Scatter(
        y=d['senkou_span_b'], 
        fill='tonexty',
        fillcolor='rgba(0, 128, 0, 0.25)',
        line=dict(color='rgba(0, 128, 0, 0.1)'),
        name='senkou_span_a above',
        **kw
    ))

    fig.add_trace(go.Scatter(y=np.where(d['senkou_span_b'] > d['senkou_span_a'], d['senkou_span_b'], d['senkou_span_a']), name='senkou_span_min', line=dict(color='#228B22', width=0), **kw))
    fig.add_trace(go.Scatter(
        y=d['senkou_span_a'], 
        fill='tonexty',
        fillcolor='rgba(128, 0, 0, 0.25)',
        line=dict(color='rgba(128, 0, 0, 0.1)'),
        name='senkou_span_b above',
        **kw
    ))

    # Add chikou_span to plot
    # fig.add_trace(go.Scatter(y=d['chikou_span'], name='chikou_span', line=dict(color='#D105F5', width=1), **kw))
    fig.update_layout(xaxis_rangeslider_visible=show_rangeslider)

    fig.update_layout(
        # title='Ichimoku',
        yaxis_title=f'{ticker_name} Price',
        xaxis_title='Date',
        legend=dict(orientation='h', y=0.1, x=0.1, yanchor='top'),
        margin=dict(t=40, b=40, r=40, l=40),
        yaxis=dict(type='log'),
        template=template,
    )
    return fig #.show()

def get_model(x_train, y_train):
    pred_model = Sequential()
    pred_model.add(LSTM(50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
    pred_model.add(LSTM(50, return_sequences=False))
    # pred_model.add(Dense(25))
    pred_model.add(Dense(25, activation='relu'))
    pred_model.add(Dense(1))
    pred_model.compile(optimizer='adam', loss='mean_squared_error')
    pred_model.fit(x_train, y_train, batch_size=32, epochs=EPOCH, verbose=0)
    return pred_model

def get_predictions(scaler, scaled_data):
    # Create the testing data set
    # Create a new array containing scaled values 
    test_data = scaled_data[training_data_len - 60: , :]
    x_test, y_test = [], dataset[training_data_len:, :]
    for i in range(60, len(test_data)):
        x_test.append(test_data[i-60:i, 0])
        
    # Convert the data to a numpy array
    x_test = np.array(x_test)
    x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1)) # Reshape the data
    # Get the models predicted price values 
    predictions = pred_model.predict(x_test)
    predictions = scaler.inverse_transform(predictions)
    rmse = np.sqrt(np.mean(((predictions - y_test) ** 2))) # root mean squared error
    return (predictions, rmse)

def get_train():
    # Scale the data
    scaler = MinMaxScaler(feature_range=(0,1))
    # scaler = StandardScaler()

    scaled_data = scaler.fit_transform(dataset)
    # Create the scaled training data set
    train_data = scaled_data[0:int(training_data_len), :]
    # Split the data into x_train and y_train data sets
    x_train, y_train = [], []
    for i in range(60, len(train_data)):
        x_train.append(train_data[i-60:i, 0])
        y_train.append(train_data[i, 0])
    # Convert the x_train and y_train to numpy arrays 
    x_train, y_train = np.array(x_train), np.array(y_train)
    # Reshape the data
    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
    return (x_train, y_train, scaler, scaled_data)

def get_data(fund):
    #Create a new dataframe with only one data column
    data = prices.filter([fund])
    #Convert the dataframe to a numpy array
    dataset = data.values
    #Get the number of rows to train the model on
    training_data_len = int(np.ceil(len(dataset) * .8))
    training_data_len
    return (data, dataset, training_data_len)

def ann_risk_return(df):
    rdf = df.agg(["mean", "std"]).T
    rdf.columns = ["Return", "Risk"]
    rdf.Return = (df.iloc[-1] / df.iloc[0]) ** (1/(((df.index[-1]-df.index[0]).days)/365)) - 1
    rdf.Risk = rdf.Risk * np.sqrt(252)
    return rdf

def get_sortino(df):
    returns = df.pct_change(1).dropna()
    downside_returns = returns[returns < 0]
    downside_std = np.std(downside_returns, axis=0)
    sortino_ratio = returns.mean() / downside_std
    return sortino_ratio

def get_parent(ticker):
    for parent in categories:
        if ticker in (categories[parent]).split():
            return parent
    return summary.loc[ticker,'category']

def h_max(s):
    try:
        is_max = s.astype('float') == s.max()
        return ['color: green' if cell else '' for cell in is_max]
    except:
        return ['' for cell in s]

def h_min(s):
    try:
        is_min= s.astype('float') == s.min()
        return ['color: blue' if cell else '' for cell in is_min]
    except:
        return ['' for cell in s]

def h_bull(s):
    try:
        is_bs = s.astype('str').str.contains('ullish') & ~s.astype('str').str.contains('earish') & ~s.astype('str').str.contains('Weak') & ~s.astype('str').str.contains(' one ')
        return ['color: green' if cell else '' for cell in is_bs]
    except:
        return ['' for cell in s]

def h_bear(s):
    try:
        is_bs = s.astype('str').str.contains('earish') & ~s.astype('str').str.contains('ullish') & ~s.astype('str').str.contains('Weak') & ~s.astype('str').str.contains(' one ')
        return ['color: red' if cell else '' for cell in is_bs]
    except:
        return ['' for cell in s]

In [ ]:
tix = " ".join(sorted(set(TEST.split())))
for _ in range(4):
    kw = {'start': start}
    end = datetime.datetime.now().strftime("%Y-%m-%d")
    idx = yf.download(tix, **kw)
    idx.index = idx.index.date
    if idx['Adj Close'].index[0] != idx['Adj Close'].dropna().index[0]:
        start = idx['Adj Close'].dropna().index[0]
        print(f"\nCommon start: {start}")
        display(idx['Adj Close'][:start].fillna(''))
        idx=idx.loc[start:]
    # FIXME: Warning: some futures ETFs show NaN for current trading day
    adj_prices = idx['Adj Close'].copy()
    if any(adj_prices.iloc[-1].isna()):
        print("Missing data from today!!!")
        columns_with_na_in_last_row = adj_prices.columns[adj_prices.isna().iloc[-1]].tolist()
        print(f"Drop {columns_with_na_in_last_row}" + 10*"\n")
        tix = adj_prices.columns[~adj_prices.isna().iloc[-1]].tolist()
    else:
        break
display(100*adj_prices.pct_change(1, fill_method=None).tail())
# idx.dropna(inplace=True) # Crypto currency trades when stock market is closed

prices = idx['Adj Close'].div(idx['Adj Close'].iloc[0]).mul(100)
summary = pd.DataFrame(index=idx['Adj Close'].columns)
tickers = {n: Ticker(n) for n in summary.index}
funds = {n: t.all_modules[n] for n, t in tickers.items()}
holdings = {fund:funds[fund]["topHoldings"]['holdings'] for fund in funds if funds[fund].get('topHoldings')}
recommendations = {n: t.recommendations for n, t in tickers.items()}
insights = {n: t.technical_insights[n] for n, t in tickers.items()}
ret = prices.pct_change().dropna()
types = []
for fund in funds:
    if 'CRYPTOCURRENCY' == funds[fund]["quoteType"]['quoteType']: # Crypto
        types.append(funds[fund]["quoteType"]['quoteType'])
    elif 'ETF' == funds[fund]["quoteType"]['quoteType']:
        types.append(funds[fund]["fundProfile"]['categoryName'])
    else: #'EQUITY' == funds[fund]["quoteType"]['quoteType'] = Stock
        types.append(funds[fund]["assetProfile"]['sector'])
summary['category'] = types

summary = pd.concat((summary, ann_risk_return(prices.div(100))), axis=1)
summary['trailingPE'] = [funds[fund]["summaryDetail"].get('trailingPE',0) for fund in funds]
summary["Sharpe"] = (summary["Return"].sub(risk_free_return))/summary["Risk"]
summary["TotalRisk"] = np.power(summary.Risk, 2)
summary["SystRisk"] = (prices.div(100).cov()*252).loc[:,'VOO' if 'VOO' in prices.columns else prices.columns[-1]]
summary["UnsystRisk"] = summary["TotalRisk"].sub(summary["SystRisk"])
json_struct = json.loads(pd.DataFrame(insights).drop(['upsell', 'symbol'], errors='ignore').T.to_json(orient="records"))
df_flat = pd.json_normalize(json_struct)
df_flat.drop([c for c in df_flat.columns if c.endswith(('indexScoreDescription', '.stateDescription', '.provider', '.indexDirection',
                                                        '.direction', '.score', '.indexScore',
        'sigDevs', 'events', 'secReports', 'projectionValuesCat', 'reports', '.reportId', '.researchReports.title', 'companySnapshot'))],
        axis=1, inplace=True)
df_flat.columns= [c.replace('instrumentInfo.', '') for c in df_flat.columns]
df_flat.columns= [c.replace('technicalEvents.', '') for c in df_flat.columns]
df_flat.columns= [c.replace('keyTechnicals.', '') for c in df_flat.columns]
df_flat.index = summary.index
df_flat = df_flat.T
df_flat = pd.concat((df_flat, pd.DataFrame(
    {fund['quoteType']['symbol']:
    {'annualReportExpenseRatio': fund.get('fundProfile',{}).get('feesExpensesInvestment',{}).get('annualReportExpenseRatio',0)}
    for fund in funds.values()}
)))
display(HTML("<style>:root { --jp-notebook-max-width: 96% !important; }</style>"))

noa = len(prices.columns)
np.random.seed(NOP)
matrix = np.random.random(noa*NOP).reshape(NOP, noa)
weights = matrix / matrix.sum(axis=1, keepdims=True)
port_ret = prices.div(100).dot(weights.T) # port_ret = ret.dot(weights.T)
port_summary = ann_risk_return(port_ret)
port_summary["Sharpe"] = (port_summary["Return"].sub(risk_free_return))/port_summary["Risk"]
port_summary["Sortino"] = get_sortino(port_ret.iloc[:,:-1])
msrp = port_summary.Sharpe.idxmax()
msrp1 = port_summary.Sortino.idxmax()
msrp_w = weights[msrp, :]
msrp1_w = weights[msrp1, :]
category = [get_parent(tick) for tick in list(prices.columns[:len(msrp_w)])]
summary['category'] = category
if len(prices.columns) == len(msrp_w):
    prices['sharpe'] = (prices * msrp_w).sum(axis=1)
if len(prices.columns) > len(msrp1_w):
    msrp1_w = np.append(msrp1_w, 0)
    prices['sortino'] = (prices * msrp1_w).sum(axis=1)
summary['weight'] = msrp_w*100
summary['weight1'] = msrp1_w[:summary.shape[0]]*100
pole = "VOO" if "VOO" in prices.columns else prices.columns[0]
summary["beta"] = summary.SystRisk / summary.loc[pole, "SystRisk"]
summary["capm_ret"] = risk_free_return + (summary.loc[pole, "Return"] - risk_free_return) * summary.beta
summary["alpha"] = summary.Return - summary.capm_ret
# summary = pd.concat((summary, df_flat.iloc[-4:].T), axis=1) # First, we add stopLoss, resistance, support...
summary = pd.concat((summary, df_flat.iloc[:-4].T), axis=1) # Next, analysts outlook
price_bands() # Calculate price bands
sortval = summary['MACD_prc'] * summary['RSI_prc']/100
edf = {}
for j, v in enumerate(sortval.sort_values(ascending=False).index):
    edf[v] = idx.swaplevel(0, 1, axis=1)[v]
    edf[v] = calc_ichimoku(edf[v])

In [ ]:
# TODO: Ichimoku Cloud signals
x=-1 # Use latest day's data
for indicator in ['MACD', 'RSI']:
    summary[f"{indicator}_prc"] = 100 - (idx[indicator][:x] < idx[indicator].iloc[x]).mean() * 100

bullish = [edf[v].loc[adj_prices.index[x]]['tenkan_sen'] > edf[v].loc[adj_prices.index[x]]['kijun_sen'] for v in adj_prices.columns]
trend = [edf[v].loc[adj_prices.index[x]]['Close'] > edf[v].loc[adj_prices.index[x]]['tenkan_sen'] > edf[v].loc[adj_prices.index[x]]['kijun_sen'] or
         edf[v].loc[adj_prices.index[x]]['Close'] < edf[v].loc[adj_prices.index[x]]['tenkan_sen'] < edf[v].loc[adj_prices.index[x]]['kijun_sen']
         for v in adj_prices.columns]
g = px.scatter(
    summary, y='RSI_prc', x='MACD_prc', hover_name=summary.index,
    color=idx['Adj Close'].pct_change().iloc[x].values*100, color_continuous_scale='RdYlGn', color_continuous_midpoint=0.0,
    title='RSI vs MACD Situational frequency', template=template, text=summary.index
)

kw = dict(
    # symbol=np.where(idx['Adj Close'].pct_change().iloc[x] > 0, 'triangle-up', 'triangle-down'),
    symbol=np.where(bullish, 'triangle-up', 'triangle-down'),
    size=(2*idx['Adj Close'].pct_change().iloc[x] / idx['Adj Close'].pct_change().std())**2+5,
    line=dict(width=3, color=idx['RSI'].iloc[x], cmax=2, cmin=-2, colorscale="RdYlGn")
)
g.update_traces(textfont=dict(color=['rgba(255, 255, 255, 0.25)' if t else 'rgba(255, 255, 255, 0.75)' for t in trend]), textposition='top center', marker=kw)#, hovertemplate=s+'=%{y}<extra></extra>'

In [ ]:
howmany = 8
x_values = sortval.sort_values(ascending=False).index[:howmany]
fig = make_subplots(rows=howmany, cols=1, subplot_titles=[f'{x}' for x in x_values])
for i, x_value in enumerate(x_values):
    single_fig = plot2_ichimoku(d=edf[x_value], ticker_name=x_value)
    for trace in single_fig.data:
        fig.add_trace(trace, col=1, row=i+1)
    fig.update_layout(xaxis={'rangeslider': {'visible': False}, 'title': {'text': 'Date'}})
    fig.update_layout(xaxis2={'rangeslider': {'visible': False}, 'title': {'text': 'Date'}})
    fig.update_layout(xaxis3={'rangeslider': {'visible': False}, 'title': {'text': 'Date'}})
    fig.update_layout(xaxis4={'rangeslider': {'visible': False}, 'title': {'text': 'Date'}})
    fig.update_layout(xaxis5={'rangeslider': {'visible': False}, 'title': {'text': 'Date'}})
    fig.update_layout(xaxis6={'rangeslider': {'visible': False}, 'title': {'text': 'Date'}})
    fig.update_layout(xaxis7={'rangeslider': {'visible': False}, 'title': {'text': 'Date'}})
    fig.update_layout(xaxis8={'rangeslider': {'visible': False}, 'title': {'text': 'Date'}})
    fig.update_layout(xaxis9={'rangeslider': {'visible': False}, 'title': {'text': 'Date'}})
    
fig.update_layout(template=template, height=400*min(howmany, len(x_values)))
fig.show()

In [ ]:
a=[]
for fund in funds:
    hold_sum = 0
    t = None
    for t in funds[fund].get('topHoldings',{}).get('holdings',{}):
        t['etf'] = fund
        t['category'] = summary.loc[fund, 'category']
        t['weight']=summary.loc[fund, 'weight' if model=='Sharpe' else 'weight1']
        t['holding'] = t['weight'] * t['holdingPercent']
        hold_sum += t['holdingPercent']
        t['Return'] = summary.loc[fund, 'Return'] #* 100
        # t['premium'] = summary.loc[fund, 'premium']
        # t['rmse'] = summary.loc[fund, 'rmse']
        # t['color'] = t['premium'] / t['rmse']
        # t['color'] = summary.loc[fund, 'Return']
        t['color'] = 2-summary.loc[fund, 'RSI_prc']/25
        a.append(t)
    z={}
    z['holdingName'] = '<other>'
    z['etf'] = fund
    z['category'] = summary.loc[fund, 'category']
    z['weight']=summary.loc[fund, 'weight' if model=='Sharpe' else 'weight1']
    z['holding'] = t['weight'] * (1-hold_sum) if t else 1
    z['Return'] = summary.loc[fund, 'Return'] #* 100
    # z['premium'] = summary.loc[fund, 'premium']
    # z['rmse'] = summary.loc[fund, 'rmse']
    z['color'] = 2-summary.loc[fund, 'MACD_prc']/25
    a.append(z)

tree = pd.DataFrame(a)
fig = px.treemap(tree, title=f"{model} Portfolio Structure", path=['category', 'etf', 'holdingName'], values='holding', #labels="parent",
         hover_name='etf',
         hover_data=['Return', 'color'],
         color_continuous_midpoint=0, color_continuous_scale="Temps", color='color', range_color=(-2,2))
fig.update_layout(margin=dict(l=10, r=10, b=10, t=30), template=template)
fig.update_traces(marker=dict(cornerradius=5))
fig.show()

In [ ]:
# TODO: chop first 26 days
titles = []
sortval = summary['MACD_prc'] * summary['RSI_prc']/100
for title in sortval.sort_values(ascending=False).index:
    titles.append("{} {:.1f}%".format(title, sortval.loc[title]))
    titles.append(None)
kw=dict(
    rows=2*summary['MACD_prc'].size,
    cols=1,
    shared_xaxes=True,
    vertical_spacing=.01,
    horizontal_spacing=.01,
    row_heights=summary['MACD_prc'].size*[300, 100],
    subplot_titles=titles
)
fig = make_subplots(**kw)
for j, v in enumerate(sortval.sort_values(ascending=False).index):
    kwtop={"row":1+2*j, "col":1}
    kwbottom={"row":2+2*j, "col":1}

    # bollinger bands
    g = px.line(edf[v], x=edf[v].index, y='Adj Close')
    g.update_traces(line_color='blue', hovertemplate='%{y:.2f}<extra></extra>')
    fig.add_trace(g.data[0], **kwtop)
    fig.add_trace(go.Scatter(
        x=pd.concat([pd.DataFrame(edf[v].dropna()),  pd.DataFrame(edf[v].dropna())[::-1]]).index,
        y=pd.concat([edf[v].dropna()['UBAND'], edf[v].dropna()['LBAND'][::-1]]),
        fill='toself',
        fillcolor='rgba(64, 64, 64, 0.35)',  # Semi-transparent green
        line=dict(color='rgba(255, 255, 255, 0)'),  # Invisible line
        name='Bollinger Bands Area',
        showlegend=False,
        hovertemplate=""
    ), **kwtop)

    # Buy/ sell signals
    for s, n in {'sell': 1, 'buy': -1}.items():
        g = px.scatter(edf[v].loc[edf[v].position_ema == n], y=edf[v].loc[edf[v].position_ema == n]['Adj Close'])
        kw = dict(symbol="triangle-up" if n==-1 else "triangle-down", line=dict(width=1, color="green" if n==-1 else "red"))
        g.update_traces(marker=kw, hovertemplate=s+'=%{y:.2f}<extra></extra>')
        fig.add_trace(g.data[0], **kwtop)
        
    ### ICHIMOKU CLOUD ###
    kwich={'x': edf[v].index, 'type': 'scatter', 'mode': 'lines', 'showlegend': False}
    fig.add_trace(go.Scatter(y=np.where(edf[v]['senkou_span_a'] > edf[v]['senkou_span_b'], edf[v]['senkou_span_a'], edf[v]['senkou_span_b']), name='senkou_span_max', line=dict(width=0), **kwich), **kwtop)
    fig.add_trace(go.Scatter(
        y=edf[v]['senkou_span_b'], 
        fill='tonexty',
        fillcolor='rgba(0, 128, 0, 0.25)',
        line=dict(color='rgba(0, 128, 0, 0.1)'),
        name='senkou_span_a above',
        **kwich), **kwtop)
    fig.add_trace(go.Scatter(y=np.where(edf[v]['senkou_span_b'] > edf[v]['senkou_span_a'], edf[v]['senkou_span_b'], edf[v]['senkou_span_a']), name='senkou_span_min', line=dict(width=0), **kwich), **kwtop)
    fig.add_trace(go.Scatter(
        y=edf[v]['senkou_span_a'], 
        fill='tonexty',
        fillcolor='rgba(128, 0, 0, 0.25)',
        line=dict(color='rgba(128, 0, 0, 0.1)'),
        name='senkou_span_b above',
        **kwich), **kwtop)

    # RSI
    g = px.line(edf[v], x=edf[v].index, y='RSI')
    g.update_traces(hovertemplate='RSI=%{y:.2f}<extra></extra>', line={'width': 1, 'color': 'orange'})
    fig.add_trace(g.data[0], **kwbottom)
    # Convergence/ divergence
    g = px.bar(edf[v], x=edf[v].index, y='MACD')
    g.update_traces(marker_color=np.where(edf[v]['MACD'] > 0, 'red', 'green'), hovertemplate='MACD=%{y:.2f}<extra></extra>')
    fig.add_trace(g.data[0], **kwbottom)
    fig.update_yaxes(type="log", **kwtop)
    fig.update_yaxes(range=[-2,2], **kwbottom)

kw = dict(
    margin=dict(l=10, r=10, b=10, t=10),
    template=template,
    height=400*j,
    hovermode='x unified',
    legend_traceorder="normal",
    spikedistance=5,
    xaxis=dict(spikecolor="white",
       spikethickness=.25,
       spikedash='dash',
       spikemode='toaxis+across+marker',
       # range=[edf[v].index[60], edf[v].index[-1]]  # Zoom in from day 26 onward
    ),
    autosize=True,
)
fig.update_layout(**kw)
fig.show();

In [ ]:
summary['url'] = summary.index
summary["url"] = summary.apply(lambda row: "<a href='https://www.etfdb.com/etf/{}/#holdings' target='_blank'>{}</a>".format(row.url, row.url), axis=1)
display(HTML(summary.infer_objects(copy=False).fillna('').T.style.apply(h_bull, axis=1).apply(h_bear, axis=1).apply(h_min, axis=1).apply(h_max, axis=1).to_html(escape=False)))

## Sharpe vs Sortino ##

In [ ]:
fig, axes = plt.subplots(1, 2, sharey=True, figsize=(16, 6))
axes[0].scatter(summary.loc[:, "Risk"], summary.loc[:, "Return"]*100, s=10, marker="x", c="lime")
axes[0].scatter(port_summary.loc[:, "Risk"], port_summary.loc[:, "Return"]*100, s=8, c=port_summary.loc[:, "Sharpe"], cmap="coolwarm",
    vmin=port_summary.Sharpe.min(), vmax=port_summary.loc[msrp].Sharpe, alpha=.7)
for i in summary.index:
    axes[0].annotate(i, xy=(summary.loc[i, "Risk"]+0.0002, summary.loc[i, "Return"]*100+.5), size=8, c="lime")

axes[1].scatter(summary.loc[:, "Risk"], summary.loc[:, "Return"]*100, s=10, marker="x", c="lime")
axes[1].scatter(port_summary.loc[:, "Risk"], port_summary.loc[:, "Return"]*100, s=8, c=port_summary.loc[:, "Sortino"], cmap="coolwarm",
    vmin=port_summary.Sortino.min(), vmax=port_summary.loc[msrp1].Sortino, alpha=.7)
for i in summary.index:
    axes[1].annotate(i, xy=(summary.loc[i, "Risk"]-0.1, summary.loc[i, "Return"]*100-1.5), size=8, c="lime")

axes[0].set_xlabel('Risk(std)')
axes[0].set_ylabel('Return')
axes[1].set_xlabel('Risk(std)')
axes[1].set_ylabel('Return')

axes[0].set_title(f"Sharpe Ratio: ({port_summary.loc[msrp].Return*100:.1f}% return {str(start)[:4]}-{end[:4]})")
axes[1].set_title(f"Sortino Ratio: ({port_summary.loc[msrp1].Return*100:.1f}% return {str(start)[:4]}-{end[:4]})")
plt.show()

In [ ]:
# Today's liquidation
s0 = 0
for v in range(100):
    ploc = int(v*(prices.shape[0]/100))
    df = (prices/prices.iloc[ploc]).dropna()
    df['Sharpe'], df['Sortino'] = (df.iloc[:,:len(msrp_w)] * msrp_w).sum(axis=1), (df.iloc[:,:len(msrp1_w)] * msrp1_w).sum(axis=1)
    ratios = (ann_risk_return(df.iloc[ploc:,-2:])['Return']*100)[-2:].values
    s0 = s0 + 1 if ratios[0] > ratios[1] else s0 -1
model = 'Sharpe' if s0 > 0 else 'Sortino'
print(f"{abs(s0)}% of samples in favor of {model}")

In [ ]:
# Range of liquidations, start prices
s0 = 0
for v in range(99, 0, -1):
    ploc = int(v * (prices.shape[0] / 100))
    if ploc < 2:
        continue
    df = (prices / prices.iloc[0]).dropna()
    df['Sharpe'], df['Sortino'] = (df.iloc[:,:len(msrp_w)] * msrp_w).sum(axis=1), (df.iloc[:,:len(msrp1_w)] * msrp1_w).sum(axis=1)
    ratios = (ann_risk_return(df.iloc[:ploc,-2:])['Return']*100)[-2:].values
    s0 = s0 + 1 if ratios[0] > ratios[1] else s0 -1
model = 'Sharpe' if s0 > 0 else 'Sortino'
print(f"{abs(s0)}% of samples in favor of {model}")

In [ ]:
fig = make_subplots(rows=1, cols=2, subplot_titles=("Price Correlation", "% Change Correlation"))#, "Price Covariance"))
kw = {
    'x':prices.columns,
    'y':prices.columns,
    'colorscale': "Jet",
    'showscale':False,
}
f1 = go.Heatmap(name='Correlation', z=prices.corr(), zmid=0.0, **kw)
f2 = go.Heatmap(name='% Changes Correlation', z=prices.pct_change().corr(), zmid=0.0, **kw)
# f3 = go.Heatmap(name='Price Covariance', z=prices.cov(), zmid=0.0, **kw)

fig.add_trace(f1, row=1, col=1)
fig.add_trace(f2, row=1, col=2)
# fig.add_trace(f3, row=1, col=3)
fig.update_layout(template=template, height=500, width=1000, margin=dict(l=10, r=10, b=10, t=20))
fig.update_yaxes(autorange="reversed")
fig.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2)
prices.index = pd.to_datetime(prices.index)
ax={'sharpe': ax1, 'sortino': ax2}
for i, v in ax.items():
    p_tr = prices[i].resample("ME").last()
    p_ret = np.log(p_tr / p_tr.shift()).dropna().to_frame()
    p_ret.columns = ["Return"]
    windows = [year for year in range(p_ret.index.size, 0, -1)]
    for period in windows:
        p_ret["#{}".format(period)] = p_ret.Return.rolling(period).mean()
    triangle = p_ret.drop(columns = ["Return"])
    triangle.index = triangle.index.to_period('M')
    sns.heatmap(triangle, annot=False, cmap="RdYlGn", vmin=-0.2/12, vmax=0.2/12, center=0, cbar=False, ax=v)

# plt.figure(figsize=(80,50))
plt.tick_params(axis = "y", labelright =True, labelleft=False, grid_alpha=.5)
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.show()

In [ ]:
v=0
log=True
df = (prices/prices.iloc[int(v*(prices.shape[0]/100))]).dropna()
ar = ann_risk_return(df.iloc[int(v*(prices.shape[0]/100)):])
layout = dict(
    title="Normalized Returns since {}. Sharpe: {:.1f}%. Sortino: {:.1f}%. Max: {:.1f} ({})".format(
        str(prices.iloc[int(v*(prices.shape[0]/100))].name)[:10],
        ar.iloc[-2]['Return']*100,
        ar.iloc[-1]['Return']*100,
        ar.Return.iloc[ar.Return.argmax()]*100,
        ar.iloc[ar.Return.argmax()].name
    ),
    log_y=True,
    template=template
)
px.line(df, **layout)

## Pricing ##

In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(x=summary.index, y=summary['SystRisk'], name='SystRisk'))
fig.add_trace(go.Bar(x=summary.index, y=summary['UnsystRisk'], name='UnsystRisk'))
fig.update_layout({
    # 'barmode': 'stack',
    'title': "Risks",
    'template': template,
    'yaxis': {'title_text': "Risk"}
})
fig

In [ ]:
options = ['alpha', 'beta', 'Risk', 'SystRisk', 'UnsystRisk', 'TotalRisk', 'Sharpe', 'capm_ret']
slider = widgets.IntSlider(min=1, max=len(options))
def print_val(v):
    titles = (f"Sharpe Portfolio Components Return vs {options[v-1]}: {port_summary.loc[msrp].Return*100:.1f}% {start.strftime('%Y')}-{end[:4]}",
            f"Sortino Portfolio Components Return vs {options[v-1]}: {port_summary.loc[msrp1].Return*100:.1f}% {start.strftime('%Y')}-{end[:4]}")
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, shared_yaxes=True, subplot_titles=titles)
    kw = {
        'hovertext': summary.index,
        'x':summary[options[v-1]],
        'y':summary['Return'],
        'mode':'markers',
        'marker':dict(size=summary['weight']+1, color=pd.factorize(summary['category'])[0]),
        'showlegend': False,
    }
    kw1 = {
        'hovertext': summary.index,
        'x':summary[options[v-1]],
        'y':summary['Return'],
        'mode':'markers',
        'marker':dict(size=summary['weight1']+1, color=pd.factorize(summary['category'])[0]),
        'showlegend': False,
    }
    f1 = go.Scatter(**kw)
    f2 = go.Scatter(**kw1)
    
    fig.add_trace(f1, row=1, col=1)
    fig.add_trace(f2, row=2, col=1)
    fig.update_layout(template=template, height=500, width=1000, margin=dict(l=10, r=10, b=10, t=30))
    return fig

interact(print_val,v=slider);

## Sectors ##

In [ ]:
# Sector coverage in funds
for i in range(summary.shape[0]):
    try: # fund_sector_weightings fails for certain funds, need to find one that succeeds
        if not any(tickers[summary.index[i]].fund_sector_weightings):
            continue
    except:
        continue
    break
snames = tickers[summary.index[i]].fund_sector_weightings.index
sectors=pd.DataFrame([
    funds[fund]["topHoldings"]['sectorWeightings'] if funds[fund].get("topHoldings",{}).get('sectorWeightings') else len(snames) * [None]
    for fund in funds
], index=summary.index, columns=snames).replace(to_replace=[None], value=[dict({'none': 0.0})])
for c in sectors.columns:
    sectors[c] = [v for d in sectors[c].values for v in d.values()]
sectors = sectors.T
sectors['sharpe']=(sectors * msrp_w).sum(axis=1)
sectors['sortino']=(sectors * msrp1_w).sum(axis=1)
sectors.sort_index(inplace=True)
px.imshow(sectors*100, template=template, title='Top Sectors')

## Overlaps ##

In [ ]:
g = net.Network(notebook=True, bgcolor='#111111', font_color='white', width="100%", height="800px")#, cdn_resources='in_line')
# g.repulsion()
g.physics = "forceAtlas2Based"
# g.inherit_edge_colors(False)
nxg=nx.complete_graph(0)
for n in summary.index: # FIXME: colors of edges
    kw = {'label':n, 'shape':'box'} #'size':abs(summary.loc[n, 'Return'])*100
    kw['title'] = textwrap.fill(funds[n]['summaryProfile'].get('longBusinessSummary', funds[n]['summaryProfile'].get('description')), 60)
    if summary.loc[n, 'Return'] < 0:
        kw['color'] = 'red'
    kw['borderWidth'] = abs(summary.loc[n]['Return']) * 100
    kw['group'] = summary.loc[n]['category']
    nxg.add_node(n, **kw)
    for h in funds[n].get('topHoldings',{}).get('holdings',{}):
        nxg.add_node(h['symbol'], label=h['holdingName'], shape='text')
        nxg.add_edge(n, h['symbol'], value=h['holdingPercent'], title=f"{100*h['holdingPercent']:.1f}%")
g.from_nx(nxg)
g.show("overlaps.html")

### Obsolete

In [ ]:
ms = range(20, 101, 10)
dfmd = pd.DataFrame(index = ms, columns=idx['Adj Close'].columns)

for finish in ms:
    n, best = min([3, len(idx['Adj Close'].columns)//2]), {c:0 for c in idx['Adj Close'].columns}
    for v in range(100):
        t0 = int(v * (prices.shape[0] * finish / 100 / 100)) # Starting time for comparison
        df = (prices.iloc[:int((prices.shape[0] * finish / 100))] / prices.iloc[t0]).dropna() # DataFrame normalized against Starting time prices
        if t0 + 1 >= df.shape[0]:
            continue
        ratios = ann_risk_return(df.iloc[int(v * (df.shape[0] / 100)):])['Return'][:-2] * 100
        for x in ratios.nlargest(n).index:
            best[x] = 1 if not x in best else best[x]+1
    summary['Top'] = [best[x] for x in sorted(best)] # Sample 100 intervals' to date for top N returns
    weight = 'weight' if model == 'Sharpe' else 'weight1'
    dfmd.loc[finish] = [best[x] for x in sorted(best)]
    
dfmd.index = [str(prices.iloc[int(prices.shape[0] * x / 100)].name)[:10] for x in range(10, 100, 10)]
px.imshow(dfmd.T, template=template, color_continuous_scale=px.colors.sequential.Viridis, color_continuous_midpoint=55,
          title=f"In Top {n} Performers {dfmd.index[0]} ... {dfmd.index[-1]}")

## Recommendations ##

In [ ]:
# Price correlation
def reg_coef(x, y, label=None, color=None, cmap=None, **kwargs):
    ax = plt.gca()
    r, _ = stats.pearsonr(x, y)
    norm = plt.Normalize(-1, 1)
    cmap = cmap if not cmap is None else plt.cm.coolwarm
    ax.annotate(f"{r:.2f}", xy=(0.5, 0.5), xycoords='axes fraction', ha='center', fontsize=16, bbox={'facecolor': cmap(norm(r)), 'alpha': r**4})
    ax.set_axis_off()

return_fig = sns.PairGrid(prices)
# Using map_upper we can specify what the upper triangle will look like.
return_fig.map_upper(plt.scatter, color='purple')
return_fig.map_upper(reg_coef, cmap=plt.get_cmap('PiYG'))

# We can also define the lower triangle in the figure, inclufing the plot type (kde) or the color map (BluePurple)
return_fig.map_lower(sns.kdeplot, cmap='cool_d')
# Finally we'll define the diagonal as a series of histogram plots of the daily return
return_fig.fig.suptitle('Price Correlation', fontsize=24)
return_fig.map_diag(plt.hist, bins=24);

---

## Screener

In [ ]:
droplist = " ".join((TEST, " ".join((categories.values()))))
shortlist = []
for r in recommendations.values():
    for v in r.values():
        if v=='No data found':
            continue
        for rec in v['recommendedSymbols']:
            if not (rec['symbol'] in droplist):
                shortlist.append(rec['symbol'])
profiles = {n:Ticker(n).fund_profile[n] for n in shortlist}
for prop in ['maxAge', 'styleBoxUrl']:
    for k,v in profiles.items():
        if prop in v:
            del v[prop]
shortlist = [p for p, v in profiles.items() if 'categoryName' in v and not 'Leveraged' in v['categoryName']]
" ".join(sorted({*shortlist}))

In [ ]:
# Recommendations
g = net.Network(notebook=True, bgcolor='#111111', font_color='white', width="100%", height="800px")#, cdn_resources='in_line')
g.repulsion()
g.physics = "forceAtlas2Based"
nxg=nx.complete_graph(0)
for n in summary.index:
    kw = {'label':n, 'shape':'box'}
    kw['group'] = summary.loc[n]['category']
    kw['title'] = textwrap.fill(funds[n]['summaryProfile'].get('longBusinessSummary', funds[n]['summaryProfile'].get('description')), 60)
    nxg.add_node(n, **kw)
    if list(recommendations[n].values()) == ['No data found']:
        continue
    for h in list(recommendations[n].values())[0]['recommendedSymbols']:
        if not h['symbol'] in shortlist:
            continue
        nxg.add_node(h['symbol'], label=h['symbol'], shape='text', title=json.dumps(profiles[h['symbol']], indent=2))
        nxg.add_edge(n, h['symbol'], value=h['score'], title=100*h['score'])
g.from_nx(nxg)
g.show("recommendations.html")

In [ ]:
s = Screener()
print([x for x in s.available_screeners if 'etf' in x and not 'asia' in x and not 'europe' in x])

In [ ]:
screener = ['fifty_two_wk_losers_etfs', 'cheapest_etfs', 'top_etfs_us'] #'top_performing_etfs', 'precious_metal_etfs'
def g(row):
    return [str(r) for r in row.values]
    
def bold_row(row):
    return ['font-weight: bold' if 'ChangePercent' in row.name else '' for v in row]

def bold_index(s):
    return '<b>{}</b>'.format(s)

def h_hi(row):
    return ['background-color: lightgreen']*len(row.values) if 'ChangePercent' in row.name else ['']*len(row.values)

droprows=["shortName", 'exchange', 'firstTradeDateMilliseconds', 'messageBoardId', 'dividendDate', 'fiftyTwoWeekLowChangePercent',
          'fiftyTwoWeekHighChangePercent', 'fiftyDayAverageChangePercent', 'twoHundredDayAverageChangePercent', 'fiftyTwoWeekLowChange',
          'fiftyTwoWeekHighChange', 'regularMarketChange', 'isEarningsDateEstimate', 'postMarketChangePercent', 'askSize', 'bidSize',
          'ipoExpectedDate', 'epsTrailingTwelveMonths', 'sharesOutstanding', 'bookValue', 'marketCap', 'netAssets']
quotes = [i for q in s.get_screeners(screener, 48).values() for i in q['quotes'] if not i['symbol'] in droplist]
df = pd.json_normalize(quotes).set_index('symbol').T
df.drop(df.index[:7], inplace=True)
df.drop([c for c in df.index if 'Time' in c], inplace=True)
df.dropna(axis=1, subset=['longName'], inplace=True)

regex = re.compile('|'.join(['everage', "Ether", 'Bitcoin', 'Bond', "Ultra", "Short", "Long", 'Hedge', 'Option', 'Covered Call', 'Treasury',
                             "Trust", "Daily", "Monthly", 'Weekly', 'Asia', 'China', 'Emerging', 'iShares MSCI']))
df.drop(columns=df.columns[df.loc['longName'].str.contains(regex)], axis=1, inplace=True)

for i in df.index:
    if 1 == len(set(g(df.loc[i,:]))):
        droprows.append(i)
df.drop(droprows, inplace=True, errors='ignore')
df = df.sort_values(by=['fiftyTwoWeekChangePercent'], axis=1, ascending=False)
df = df.fillna('')

df.drop(droplist, axis=1, errors='ignore', inplace=True)
df.head(60).style.apply(h_max, axis=1).apply(h_min, axis=1).apply(bold_row, axis=1).apply(h_hi, axis=1)

In [ ]:
" ".join(df.columns)

## Algo Return

In [ ]:
algo=(1 + idx['Adj Close'].pct_change() * idx['position_ema'].shift(1)).cumprod()
px.line(algo, template=template, log_y=True, title="Algo EMA")

In [ ]:
algo=(1 + idx['Adj Close'].pct_change() * idx['position_rsi'].shift(1)).cumprod()
px.line(algo, template=template, log_y=True, title="Algo RSI")

In [ ]:
idx.columns

In [ ]:
# idx = calc_ichimoku(idx)
v = 'CRPT'
edf[v]

In [ ]:
def calc_positions_ichimoku(d):
    global idx
    columns = d['Low'].columns
    # Adding signals for buy/ sell
    condition = np.where(
        (d['tenkan_sen'] > d['kijun_sen']) & (d['Close'] > d['tenkan_sen']),
        1, 0
    )
    sgn = pd.DataFrame(condition, columns=columns, index=d.index)
    sgn.columns = pd.MultiIndex.from_product([['signal_ichimoku'], columns])
    pos = sgn.diff().fillna(0)
    pos.columns = pd.MultiIndex.from_product([['position_ichimoku'], columns])
    idx.drop(columns='signal_ichimoku', inplace=True, errors='ignore')
    idx.drop(columns='position_ichimoku', inplace=True, errors='ignore')
    idx = idx.join(pos)
    idx = idx.join(sgn)
calc_positions_ichimoku(idx)

algo=(1 + idx['Adj Close'].pct_change() * idx['position_ichimoku'].shift(1)).cumprod()
px.line(algo, template=template, log_y=True, title="Algo Ichimoku")

In [ ]:
signals = ['signal_ema', 'signal_rsi', 'signal_ichimoku']
idx[signals][:].swaplevel(0, 1, axis=1)['CRPT'].tail(60)

## Timestamp

In [ ]:
datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")